In [1]:
from histolab.data import prostate_tissue, ovarian_tissue, ihc_kidney, ihc_breast, heart_tissue, breast_tissue_diagnostic_red_pen, breast_tissue_diagnostic_green_pen, breast_tissue_diagnostic_black_pen, breast_tissue, aorta_tissue

In [11]:
!pip install pooch

In [4]:
prostate_svs, prostate_path = prostate_tissue()

In [5]:
ovarian_svs, ovarian_path = ovarian_tissue()

In [6]:
ihc_kidney_svs, ihc_kidney_path = ihc_kidney()

ValueError: SHA256 hash of downloaded file (?format=tif) does not match the known hash: expected 8a4318ac713b4cf50c3314760da41ab7653e10e90531ecd0c787f1386857a4ef but got a5b5421be549f1b034c63847942ca41e14079000007ecbf454bd5a4683f56680. Deleted download for safety. The downloaded file may have been corrupted or the known hash may be outdated.

In [7]:
ihc_breast_svs, ihc_breast_path = ihc_breast()

ValueError: SHA256 hash of downloaded file (?format=tif) does not match the known hash: expected 7db49ff9fc3f6022ae334cf019e94ef4450f7d4cf0d71783e0f6ea82965d3a52 but got 0628dfb9e2aa1af5f84692bdd9fb3d81e021cde8dd26d831c124f7975212a246. Deleted download for safety. The downloaded file may have been corrupted or the known hash may be outdated.

In [8]:
heart_svs, heart_path = heart_tissue()

In [ ]:
breast_diagnostic_red_pen_svs, breast_diagnostic_red_pen_path = breast_tissue_diagnostic_red_pen()


In [9]:
breast_diagnostic_green_pen_svs, breast_diagnostic_green_pen_path = breast_tissue_diagnostic_green_pen()


In [10]:
breast_diagnostic_black_pen_svs, breast_diagnostic_black_pen_path = breast_tissue_diagnostic_black_pen()


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
breast_svs, breast_path = breast_tissue()


In [ ]:
aorta_svs, aorta_path = aorta_tissue()

In [11]:
import requests
import json
import os
from pathlib import Path

GDC_API_ENDPOINT = "https://api.gdc.cancer.gov"
DATA_DIR = "tcga_brca_slides"
METADATA_DIR = os.path.join(DATA_DIR, "metadata")

def create_directories():
    Path(DATA_DIR).mkdir(exist_ok=True)
    Path(METADATA_DIR).mkdir(exist_ok=True)

def get_manifest(project_id="TCGA-BRCA"):
    url = f"{GDC_API_ENDPOINT}/files"
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
                {"op": "=", "content": {"field": "data_category", "value": "Biospecimen"}},
                {"op": "=", "content": {"field": "data_type", "value": "Slide Image"}}
            ]
        }),
        "fields": "file_id,file_name,md5sum,case_id",
        "format": "json",
        "size": 2000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def download_file(file_id, file_name, md5sum):
    url = f"{GDC_API_ENDPOINT}/data/{file_id}"
    output_path = os.path.join(DATA_DIR, file_name)
    if os.path.exists(output_path):
        print(f"Skipping {file_name}, already exists")
        return
    response = requests.get(url)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_name}")

def save_metadata(file_id, metadata):
    output_path = os.path.join(METADATA_DIR, f"{file_id}.json")
    with open(output_path, "w") as f:
        json.dump(metadata, f, indent=2)

def main():
    create_directories()
    manifest = get_manifest()
    files = manifest["data"]["hits"]
    
    for file in files:
        file_id = file["file_id"]
        file_name = file["file_name"]
        md5sum = file["md5sum"]
        download_file(file_id, file_name, md5sum)
        save_metadata(file_id, file)

if __name__ == "__main__":
    main()

Downloaded TCGA-C8-A27B-01A-01-TSA.b93af347-1b83-4284-afaf-b3c14d8e038b.svs


KeyboardInterrupt: 

In [12]:
import requests
import json
import os
from pathlib import Path

GDC_API_ENDPOINT = "https://api.gdc.cancer.gov"
METADATA_DIR = "tcga_brca_metadata"

def create_directories():
    Path(METADATA_DIR).mkdir(exist_ok=True)

def get_manifest(project_id="TCGA-BRCA"):
    url = f"{GDC_API_ENDPOINT}/files"
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
                {"op": "=", "content": {"field": "data_category", "value": "Biospecimen"}},
                {"op": "=", "content": {"field": "data_type", "value": "Slide Image"}}
            ]
        }),
        "fields": "file_id,file_name,md5sum,case_id,file_size,data_format",
        "format": "json",
        "size": 2000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def save_metadata(file_id, metadata):
    output_path = os.path.join(METADATA_DIR, f"{file_id}.json")
    with open(output_path, "w") as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved metadata for {file_id}")

def summarize_files(files):
    total_files = len(files)
    total_size_bytes = sum(file.get("file_size", 0) for file in files)
    total_size_mb = total_size_bytes / (1024 * 1024)
    formats = set(file.get("data_format", "Unknown") for file in files)
    
    print(f"\nSummary:")
    print(f"Total number of slide images: {total_files}")
    print(f"Total size of slides: {total_size_mb:.2f} MB")
    print(f"File formats: {', '.join(formats)}")

def main():
    create_directories()
    manifest = get_manifest()
    files = manifest["data"]["hits"]
    
    for file in files:
        file_id = file["file_id"]
        save_metadata(file_id, file)
    
    summarize_files(files)

if __name__ == "__main__":
    main()

Saved metadata for 4781996e-3d01-4a40-bb3a-3ef6e535dcae
Saved metadata for 0fb0d10a-9cac-4499-b512-9e6fb607bd15
Saved metadata for aa3f0051-ea7f-47f2-a7aa-315c1bdc3f18
Saved metadata for a779fff9-b54a-4c18-936f-43f75119b9ad
Saved metadata for b052b2c3-fa18-4706-92e3-8137af39226c
Saved metadata for 4ff19ed9-4ae2-4a7f-b7d0-cad20dc75483
Saved metadata for e5806a73-54c4-43d1-a44b-db5f43c8e832
Saved metadata for 2449ff02-6925-4f25-9074-7c5fbeab0bd2
Saved metadata for 03a7acd9-178a-4165-9588-fa63e46f3387
Saved metadata for 07a95a70-bea1-4ab8-9fbc-01b00b79b351
Saved metadata for 86867852-64fe-43e6-894d-b53b146c3518
Saved metadata for b73ca1e9-4938-4467-a805-fefc230b543b
Saved metadata for 8daf3c99-e481-4c05-bd7e-30bf7529b530
Saved metadata for 1bac9786-4d56-4f08-9792-ed131b398907
Saved metadata for b413ae9d-cb96-4824-8f26-a3654d52da82
Saved metadata for 3495efa5-0159-491f-a4f6-42fd9f617772
Saved metadata for 3f3a6de8-c315-47b9-ac06-65750905c290
Saved metadata for e6fe1a97-4a48-4da4-bb2f-d50e0

In [15]:
import requests
import json
import os
from pathlib import Path
from collections import defaultdict

GDC_API_ENDPOINT = "https://api.gdc.cancer.gov"
METADATA_DIR = "tcga_brca_metadata_by_patient"

def create_directories():
    Path(METADATA_DIR).mkdir(exist_ok=True)

def get_manifest(project_id="TCGA-BRCA"):
    url = f"{GDC_API_ENDPOINT}/files"
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
                {"op": "=", "content": {"field": "data_category", "value": "Biospecimen"}},
                {"op": "=", "content": {"field": "data_type", "value": "Slide Image"}}
            ]
        }),
        "fields": "file_id,file_name,md5sum,case_id,file_size,data_format,experimental_strategy,cases.submitter_id",
        "format": "json",
        "size": 2000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def group_by_patient(files):
    patient_slides = defaultdict(list)
    for file in files:
        case_id = file.get("case_id")
        submitter_id = file.get("cases", [{}])[0].get("submitter_id", "Unknown")
        identifier = case_id or submitter_id
        patient_slides[identifier].append(file)
    return patient_slides

def save_metadata(identifier, slides):
    output_path = os.path.join(METADATA_DIR, f"{identifier}.json")
    with open(output_path, "w") as f:
        json.dump(slides, f, indent=2)
    print(f"Saved metadata for patient {identifier} ({len(slides)} slides)")

def summarize_files(patient_slides):
    total_files = sum(len(slides) for slides in patient_slides.values())
    total_size_bytes = sum(file.get("file_size", 0) for slides in patient_slides.values() for file in slides)
    total_size_mb = total_size_bytes / (1024 * 1024)
    formats = set(file.get("data_format", "Unknown") for slides in patient_slides.values() for file in slides)
    
    print(f"\nSummary:")
    print(f"Total number of patients: {len(patient_slides)}")
    print(f"Total number of slide images: {total_files}")
    print(f"Total size of slides: {total_size_mb:.2f} MB")
    print(f"File formats: {', '.join(formats)}")
    print("Slides per patient:")
    for identifier, slides in sorted(patient_slides.items()):
        patient_size_mb = sum(file.get("file_size", 0) for file in slides) / (1024 * 1024)
        print(f"  Patient {identifier}: {len(slides)} slides, {patient_size_mb:.2f} MB")
    print("Note: Slide dimensions (width, height) are not available in GDC metadata. Download SVS files and use OpenSlide to extract dimensions.")

def main():
    create_directories()
    manifest = get_manifest()
    files = manifest["data"]["hits"]
    patient_slides = group_by_patient(files)
    
    for identifier, slides in patient_slides.items():
        save_metadata(identifier, slides)
    
    summarize_files(patient_slides)

if __name__ == "__main__":
    main()

Saved metadata for patient TCGA-C8-A27B (2 slides)
Saved metadata for patient TCGA-E9-A1R0 (2 slides)
Saved metadata for patient TCGA-BH-A0C1 (3 slides)
Saved metadata for patient TCGA-EW-A1P1 (2 slides)
Saved metadata for patient TCGA-EW-A1IZ (2 slides)
Saved metadata for patient TCGA-OL-A5D6 (2 slides)
Saved metadata for patient TCGA-E9-A5FL (2 slides)
Saved metadata for patient TCGA-AC-A23G (3 slides)
Saved metadata for patient TCGA-AO-A1KO (2 slides)
Saved metadata for patient TCGA-D8-A27P (2 slides)
Saved metadata for patient TCGA-AR-A0TZ (3 slides)
Saved metadata for patient TCGA-B6-A0RH (3 slides)
Saved metadata for patient TCGA-AC-A2QI (2 slides)
Saved metadata for patient TCGA-D8-A1JH (2 slides)
Saved metadata for patient TCGA-A8-A06R (3 slides)
Saved metadata for patient TCGA-E2-A152 (2 slides)
Saved metadata for patient TCGA-E9-A3QA (2 slides)
Saved metadata for patient TCGA-BH-A0AW (3 slides)
Saved metadata for patient TCGA-A7-A6VY (2 slides)
Saved metadata for patient TCGA

In [ ]:
import requests
import json
import os
from pathlib import Path
from collections import defaultdict

download_slides = False

GDC_API_ENDPOINT = "https://api.gdc.cancer.gov"
BASE_DIR = "tcga_brca_data"
METADATA_DIR = os.path.join(BASE_DIR, "metadata")
SLIDES_DIR = os.path.join(BASE_DIR, "slides")

def create_directories():
    Path(METADATA_DIR).mkdir(parents=True, exist_ok=True)
    if download_slides:
        Path(SLIDES_DIR).mkdir(parents=True, exist_ok=True)

def get_manifest(project_id="TCGA-BRCA"):
    url = f"{GDC_API_ENDPOINT}/files"
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
                {"op": "=", "content": {"field": "data_category", "value": "Biospecimen"}},
                {"op": "=", "content": {"field": "data_type", "value": "Slide Image"}}
            ]
        }),
        "fields": "file_id,file_name,md5sum,case_id,file_size,data_format,experimental_strategy,cases.submitter_id",
        "format": "json",
        "size": 2000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def group_by_patient(files):
    patient_slides = defaultdict(list)
    for file in files:
        case_id = file.get("case_id")
        submitter_id = file.get("cases", [{}])[0].get("submitter_id", "Unknown")
        identifier = case_id or submitter_id
        patient_slides[identifier].append(file)
    return patient_slides

def save_metadata(identifier, slides):
    output_path = os.path.join(METADATA_DIR, f"{identifier}.json")
    with open(output_path, "w") as f:
        json.dump(slides, f, indent=2)
    print(f"Saved metadata for patient {identifier} ({len(slides)} slides)")

def download_file(file_id, file_name, identifier, md5sum):
    patient_dir = os.path.join(SLIDES_DIR, identifier)
    Path(patient_dir).mkdir(exist_ok=True)
    output_path = os.path.join(patient_dir, file_name)
    if os.path.exists(output_path):
        print(f"Skipping {file_name} for patient {identifier}, already exists")
        return
    url = f"{GDC_API_ENDPOINT}/data/{file_id}"
    response = requests.get(url)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_name} for patient {identifier}")

def summarize_files(patient_slides):
    total_files = sum(len(slides) for slides in patient_slides.values())
    total_size_bytes = sum(file.get("file_size", 0) for slides in patient_slides.values() for file in slides)
    total_size_mb = total_size_bytes / (1024 * 1024)
    formats = set(file.get("data_format", "Unknown") for slides in patient_slides.values() for file in slides)
    
    print(f"\nSummary:")
    print(f"Total number of patients: {len(patient_slides)}")
    print(f"Total number of slide images: {total_files}")
    print(f"Total size of slides: {total_size_mb:.2f} MB")
    print(f"File formats: {', '.join(formats)}")
    print("Slides per patient:")
    for identifier, slides in sorted(patient_slides.items()):
        patient_size_mb = sum(file.get("file_size", 0) for file in slides) / (1024 * 1024)
        print(f"  Patient {identifier}: {len(slides)} slides, {patient_size_mb:.2f} MB")



In [18]:
create_directories()
manifest = get_manifest()
files = manifest["data"]["hits"]
patient_slides = group_by_patient(files)

for identifier, slides in patient_slides.items():
    save_metadata(identifier, slides)
    if download_slides:
        for file in slides:
            file_id = file["file_id"]
            file_name = file["file_name"]
            md5sum = file["md5sum"]
            download_file(file_id, file_name, identifier, md5sum)

summarize_files(patient_slides)

Saved metadata for patient TCGA-C8-A27B (2 slides)
Saved metadata for patient TCGA-E9-A1R0 (2 slides)
Saved metadata for patient TCGA-BH-A0C1 (3 slides)
Saved metadata for patient TCGA-EW-A1P1 (2 slides)
Saved metadata for patient TCGA-EW-A1IZ (2 slides)
Saved metadata for patient TCGA-OL-A5D6 (2 slides)
Saved metadata for patient TCGA-E9-A5FL (2 slides)
Saved metadata for patient TCGA-AC-A23G (3 slides)
Saved metadata for patient TCGA-AO-A1KO (2 slides)
Saved metadata for patient TCGA-D8-A27P (2 slides)
Saved metadata for patient TCGA-AR-A0TZ (3 slides)
Saved metadata for patient TCGA-B6-A0RH (3 slides)
Saved metadata for patient TCGA-AC-A2QI (2 slides)
Saved metadata for patient TCGA-D8-A1JH (2 slides)
Saved metadata for patient TCGA-A8-A06R (3 slides)
Saved metadata for patient TCGA-E2-A152 (2 slides)
Saved metadata for patient TCGA-E9-A3QA (2 slides)
Saved metadata for patient TCGA-BH-A0AW (3 slides)
Saved metadata for patient TCGA-A7-A6VY (2 slides)
Saved metadata for patient TCGA

In [2]:
import requests
import json
import os
from pathlib import Path
from collections import defaultdict
import csv

# Set this to True to download slides, False for metadata only
download_slides = True

GDC_API_ENDPOINT = "https://api.gdc.cancer.gov"
BASE_DIR = "tcga_data"
METADATA_DIR = os.path.join(BASE_DIR, "metadata")
SLIDES_DIR = os.path.join(BASE_DIR, "slides")
# PROJECTS = [
#     "TCGA-BRCA", "TCGA-LUAD", "TCGA-LUSC", "TCGA-OV", "TCGA-GBM",
#     "TCGA-UCEC", "TCGA-COAD", "TCGA-PRAD", "TCGA-THCA", "TCGA-HNSC", "TCGA-LIHC"
# ]

PROJECTS = ["TCGA-OV"]

def create_directories(project_id):
    project_metadata_dir = os.path.join(METADATA_DIR, project_id)
    Path(project_metadata_dir).mkdir(parents=True, exist_ok=True)
    if download_slides:
        project_slides_dir = os.path.join(SLIDES_DIR, project_id)
        Path(project_slides_dir).mkdir(parents=True, exist_ok=True)
    return project_metadata_dir, project_slides_dir if download_slides else None

def get_manifest(project_id):
    url = f"{GDC_API_ENDPOINT}/files"
    params = {
        "filters": json.dumps({
            "op": "and",
            "content": [
                {"op": "=", "content": {"field": "cases.project.project_id", "value": project_id}},
                {"op": "=", "content": {"field": "data_category", "value": "Biospecimen"}},
                {"op": "=", "content": {"field": "data_type", "value": "Slide Image"}}
            ]
        }),
        "fields": "file_id,file_name,md5sum,case_id,file_size,data_format,experimental_strategy,cases.submitter_id",
        "format": "json",
        "size": 10000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def group_by_patient(files):
    patient_slides = defaultdict(list)
    for file in files:
        case_id = file.get("case_id")
        submitter_id = file.get("cases", [{}])[0].get("submitter_id", "Unknown")
        identifier = case_id or submitter_id
        patient_slides[identifier].append(file)
    return patient_slides

def save_metadata(project_id, identifier, slides, project_metadata_dir):
    output_path = os.path.join(project_metadata_dir, f"{identifier}.json")
    with open(output_path, "w") as f:
        json.dump(slides, f, indent=2)
    print(f"Saved metadata for {project_id}, patient {identifier} ({len(slides)} slides)")

def download_file(project_id, file_id, file_name, identifier, md5sum, project_slides_dir):
    patient_dir = os.path.join(project_slides_dir, identifier)
    Path(patient_dir).mkdir(exist_ok=True)
    output_path = os.path.join(patient_dir, file_name)
    if os.path.exists(output_path):
        print(f"Skipping {file_name} for {project_id}, patient {identifier}, already exists")
        return
    url = f"{GDC_API_ENDPOINT}/data/{file_id}"
    response = requests.get(url)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_name} for {project_id}, patient {identifier}")

def generate_project_summary_csv(project_id, patient_slides):
    csv_path = os.path.join(BASE_DIR, f"{project_id}_summary.csv")
    total_files = sum(len(slides) for slides in patient_slides.values())
    total_size_bytes = sum(file.get("file_size", 0) for slides in patient_slides.values() for file in slides)
    total_size_mb = total_size_bytes / (1024 * 1024)
    formats = set(file.get("data_format", "Unknown") for slides in patient_slides.values() for file in slides)
    tissue_slides = sum(1 for slides in patient_slides.values() for file in slides if file.get("experimental_strategy") == "Tissue Slide")
    diagnostic_slides = sum(1 for slides in patient_slides.values() for file in slides if file.get("experimental_strategy") == "Diagnostic Slide")
    
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Project", "Total Patients", "Total Slides", "Tissue Slides", "Diagnostic Slides", "Total Size (MB)", "File Formats"])
        writer.writerow([project_id, len(patient_slides), total_files, tissue_slides, diagnostic_slides, f"{total_size_mb:.2f}", ", ".join(formats)])
        writer.writerow([])
        writer.writerow(["Patient ID", "Number of Slides", "Tissue Slides", "Diagnostic Slides", "Size (MB)"])
        for identifier, slides in sorted(patient_slides.items()):
            patient_size_mb = sum(file.get("file_size", 0) for file in slides) / (1024 * 1024)
            patient_tissue = sum(1 for file in slides if file.get("experimental_strategy") == "Tissue Slide")
            patient_diagnostic = sum(1 for file in slides if file.get("experimental_strategy") == "Diagnostic Slide")
            writer.writerow([identifier, len(slides), patient_tissue, patient_diagnostic, f"{patient_size_mb:.2f}"])
    
    print(f"\nGenerated project summary CSV for {project_id}: {csv_path}")
    print(f"Summary for {project_id}:")
    print(f"Total number of patients: {len(patient_slides)}")
    print(f"Total number of slide images: {total_files}")
    print(f"Tissue slides: {tissue_slides}")
    print(f"Diagnostic slides: {diagnostic_slides}")
    print(f"Total size of slides: {total_size_mb:.2f} MB")
    print(f"File formats: {', '.join(formats)}")
    print("Note: Slide dimensions (width, height) are not available in GDC metadata. Download SVS files and use OpenSlide to extract dimensions.")
    
    return {
        "project": project_id,
        "total_patients": len(patient_slides),
        "total_slides": total_files,
        "tissue_slides": tissue_slides,
        "diagnostic_slides": diagnostic_slides,
        "total_size_mb": total_size_mb,
        "file_formats": ", ".join(formats)
    }

def generate_all_projects_summary_csv(project_summaries):
    csv_path = os.path.join(BASE_DIR, "all_tcga_projects_summary.csv")
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Project", "Total Patients", "Total Slides", "Tissue Slides", "Diagnostic Slides", "Total Size (MB)", "File Formats"])
        for summary in project_summaries:
            writer.writerow([
                summary["project"],
                summary["total_patients"],
                summary["total_slides"],
                summary["tissue_slides"],
                summary["diagnostic_slides"],
                f"{summary['total_size_mb']:.2f}",
                summary["file_formats"]
            ])
    print(f"\nGenerated all projects summary CSV: {csv_path}")

def main():
    project_summaries = []
    for project_id in PROJECTS:
        print(f"\nProcessing {project_id}...")
        project_metadata_dir, project_slides_dir = create_directories(project_id)
        manifest = get_manifest(project_id)
        files = manifest["data"]["hits"]
        patient_slides = group_by_patient(files)
        
        for identifier, slides in patient_slides.items():
            save_metadata(project_id, identifier, slides, project_metadata_dir)
            if download_slides:
                for file in slides:
                    file_id = file["file_id"]
                    file_name = file["file_name"]
                    md5sum = file["md5sum"]
                    download_file(project_id, file_id, file_name, identifier, md5sum, project_slides_dir)
        
        project_summary = generate_project_summary_csv(project_id, patient_slides)
        project_summaries.append(project_summary)
    
    generate_all_projects_summary_csv(project_summaries)

main()


Processing TCGA-OV...
Saved metadata for TCGA-OV, patient TCGA-29-1705 (4 slides)
Downloaded TCGA-29-1705-01A-01-BS1.febc5de5-d46e-4d34-a597-501782af363a.svs for TCGA-OV, patient TCGA-29-1705
Downloaded TCGA-29-1705-02A-01-TS1.4d5e4d51-e4ba-4d62-925e-3a3c302c9195.svs for TCGA-OV, patient TCGA-29-1705


KeyboardInterrupt: 